## Merge csv

In [1]:
import pandas as pd

In [4]:
# read all csv and merge them
import os

path = "./data/channel_list"
files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, files))
# unique values
df = df.drop_duplicates()

In [6]:
df = df[df['n_subs'] > 10000]

In [7]:
len(df)

1855

In [8]:
df.to_csv('10k_sub_+.csv', index=False)

## Out.txt

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/channel_list_merged/10k_sub_+.csv")

In [3]:
len(df[df['n_subs'] > 10000])

736

In [27]:
rs = []
import json
with open("data/channel_quality/all.jsonl", "r") as f:
    r = f.readlines()
    for i in r:
        # rs.append(json.loads(i.replace("'", "\"").replace("False", "false").replace("True", "true")))
        rs.append(json.loads(i))

In [24]:
# rs[0]

In [37]:
import numpy as np

def clean_channel_cond(r):
    n_total = len(r['snrss'])
    n_snr_ok = (np.array(r['snrss']) > 20).sum()
    # n_ac_ok = (np.array(r['acss'])).sum()
    ac_speech_probs = []
    for ac_seg in r['acss']:
        for item in ac_seg:
            if item['label'] == 'Speech':
                ac_speech_probs.append(item['score'])
    n_ac_ok = (np.array(ac_speech_probs) > 0.9).sum()
    return n_total > 0 and n_snr_ok / n_total > 0.4 and n_ac_ok / n_total > 0.3

def get_num(r):
    n_total = len(r['snrss'])
    # n_snr_ok = (np.array(r['snrss']) > 15).sum()
    n_snr_ok = (np.array(r['snrss']) > 20).sum()
    # n_ac_ok = (np.array(r['acss'])).sum()
    ac_speech_probs = []
    for ac_seg in r['acss']:
        for item in ac_seg:
            if item['label'] == 'Speech':
                ac_speech_probs.append(item['score'])
    n_ac_ok = (np.array(ac_speech_probs) > 0.9).sum()
    return {"URL": r['url'], "n_total": n_total, "n_snr_ok": n_snr_ok, "n_ac_ok": n_ac_ok}

r_ok = []
n_ok = []
for r in rs:
    if clean_channel_cond(r):
        r_ok.append(r)
        n_ok.append(get_num(r))

r_not_ok = []
n_not_ok = []
for r in rs:
    if not clean_channel_cond(r):
        r_not_ok.append(r)
        n_not_ok.append(get_num(r))

In [38]:
len(r_ok)

16

In [39]:
len(r_not_ok)

353

In [40]:
ok_df = df[df['url'].isin([r['url'] for r in r_ok])]
not_ok_df = df[df['url'].isin([r['url'] for r in r_not_ok])]

In [32]:
# ok_df

In [43]:
ok_df.to_csv('tmp_16_channel_ok.csv', index=False)

In [33]:
# join n_total, n_snr_ok, n_ac_ok
ok_df = pd.merge(ok_df, pd.DataFrame(n_ok), left_on='URL', right_on='URL')
not_ok_df = pd.merge(not_ok_df, pd.DataFrame(n_not_ok), left_on='URL', right_on='URL')

KeyError: 'URL'

In [44]:
ok_df.to_csv('channels_10k_sub+_first_200_clean.csv', index=False)
not_ok_df.to_csv('channels_10k_sub+_first_200_not_clean.csv', index=False)